In [92]:
# Load the autoreload extension
%load_ext autoreload

# Set autoreload mode to automatically reload all modules before execution
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
import pandas as pd
from sqlalchemy.orm import Session
from sql_db.database import SessionLocal
from sql_db.models import HighSchoolType, Category, UCCampus
from sql_db import models
from sql_db.process_csv_file import add_ethnicity_csv_file_to_db, add_gender_csv_file_to_db, add_gpa_csv_file_to_db
import sys
import os
from routes import get_files_dataframe

In [97]:
# Define the path you want to add
new_path = os.path.abspath('/app/app')

# Add the new path to the PYTHONPATH
if new_path not in sys.path:
    sys.path.append(new_path)


In [99]:
df_files = get_files_dataframe()

hello world


In [100]:
df_files.head(20)

,id,location,high_school_type,uc_campus_id,category,year,is_added_to_db
0,245,/app/app/handle-files/files/FR GPA by Year Ber...,HighSchoolType.ALL,2,Category.GPA,2023,False
1,246,/app/app/handle-files/files/FR ENR Eth non-CA ...,HighSchoolType.NON_CA,4,Category.ETHNICITY,2023,False
2,247,/app/app/handle-files/files/FR ENR Eth Foreign...,HighSchoolType.FOREIGN,8,Category.ETHNICITY,2023,False
3,248,/app/app/handle-files/files/FR ENR Eth non-CA ...,HighSchoolType.NON_CA,9,Category.ETHNICITY,2023,False
4,251,/app/app/handle-files/files/FR GPA by Year Irv...,HighSchoolType.ALL,5,Category.GPA,2023,False
5,252,/app/app/handle-files/files/FR GPA by Year Mer...,HighSchoolType.ALL,8,Category.GPA,2023,False
6,254,/app/app/handle-files/files/FR ENR Eth CA Publ...,HighSchoolType.CA_PUBLIC,1,Category.ETHNICITY,2023,False
7,255,/app/app/handle-files/files/FR ENR Eth CA Priv...,HighSchoolType.CA_PRIVATE,2,Category.ETHNICITY,2023,False
8,256,/app/app/handle-files/files/FR ENR Eth CA Priv...,HighSchoolType.CA_PRIVATE,4,Category.ETHNICITY,2023,False
9,257,/app/app/handle-files/files/FR ENR Eth CA Priv...,HighSchoolType.CA_PRIVATE,7,Category.ETHNICITY,2023,False


In [146]:
db.close()

In [147]:

# Create a database session
db = SessionLocal()

In [237]:
i = 0
for _, row in df_files.iterrows():

    # i += 1
    # if i > 10: break

    uc_campus_id = int(row['uc_campus_id'])
    uc_campus = db.query(UCCampus).filter(UCCampus.id == uc_campus_id).first()
    uc_campus_name = uc_campus.campus_name if uc_campus else "Unknown"
    # Convert HighSchoolType, uc_campus, and category to their text representations
    high_school_type = HighSchoolType(row['high_school_type']).value
    category = Category(row['category']).value
    file_path = row['location']
    year = int(row['year'])
    is_added_to_db = row['is_added_to_db']

    
    # # if has been processed 
    # if is_added_to_db: continue




    print(f"Processing row:\n"
          f"UC Campus: {uc_campus_name}\n"
          f"High School Type: {high_school_type}\n"
          f"Category: {category}\n"
          f"File Path: {file_path}\n"
          f"Year: {year}\n"
          f"Already in DB: {is_added_to_db}")


    if category=='Ethnicity':
        #  result = add_ethnicity_csv_file_to_db(db, file_path, uc_campus_name, year, high_school_type)
        pass
    elif category=='GPA':                 
        result = add_gpa_csv_file_to_db(db, file_path, uc_campus_name, year)
    elif category=='Gender':
        # result = add_gender_csv_file_to_db(db, file_path, uc_campus_name, year)
        pass
    else:
        result = False

    if result:
        print("File content successfully inserted into the database.")

        file = db.query(models.File).filter_by(location=file_path).first()
        if file:
            file.is_added_to_db = True
            db.commit()
    else:
        print("Failed to insert file content into the database.")




Processing row:
UC Campus: Berkeley
High School Type: All
Category: GPA
File Path: /app/app/handle-files/files/FR GPA by Year Berkeley 2023.csv
Year: 2023
Already in DB: False
File content successfully inserted into the database.
Processing row:
UC Campus: San Diego
High School Type: non_CA
Category: Ethnicity
File Path: /app/app/handle-files/files/FR ENR Eth non-CA UCSD 2023.csv
Year: 2023
Already in DB: False
File content successfully inserted into the database.
Processing row:
UC Campus: Merced
High School Type: Foreign
Category: Ethnicity
File Path: /app/app/handle-files/files/FR ENR Eth Foreign Merced 2023.csv
Year: 2023
Already in DB: False
File content successfully inserted into the database.
Processing row:
UC Campus: Santa Cruz
High School Type: non_CA
Category: Ethnicity
File Path: /app/app/handle-files/files/FR ENR Eth non-CA UCSC 2023.csv
Year: 2023
Already in DB: False
File content successfully inserted into the database.
Processing row:
UC Campus: Irvine
High School Type:

## Get high school data

In [238]:
db.close()

In [239]:
db=SessionLocal()

In [240]:
high_school_data = crud.get_high_school_data(db, "Woodbridge High School", "Irvine")
if high_school_data:
    print(high_school_data)
else:
    print("High school not found")

{'high_school': {'id': 38334, 'uc_school_name': 'WOODBRIDGE HIGH SCHOOL51274', 'school_name': 'WOODBRIDGE HIGH SCHOOL', 'city': 'Irvine', 'county': 'Orange', 'state': 'CA', 'country': 'United States', 'is_public': True}, 'Los Angeles': {'App': {'All': 202, 'African American': 7, 'Hispanic/ Latinx': 22, 'Asian': 115, 'White': 44, 'Domestic Unknown': 9, 'total_applicants': 202, 'male_applicants': 90, 'female_applicants': 104, 'other_applicants': 0, 'unknown_gender': 0, 'Mean GPA': 4.06}, 'Adm': {'All': 25, 'Asian': 15, 'White': 5, 'total_applicants': 25, 'male_applicants': 9, 'female_applicants': 14, 'other_applicants': 0, 'unknown_gender': 0, 'Mean GPA': 4.29}, 'Enr': {'All': 10, 'Asian': 8, 'total_applicants': 10, 'male_applicants': 3, 'female_applicants': 7, 'other_applicants': 0, 'unknown_gender': 0, 'Mean GPA': 4.28}}, 'Berkeley': {'App': {'All': 165, 'Hispanic/ Latinx': 15, 'Asian': 101, 'White': 33, 'Domestic Unknown': 10, 'total_applicants': 165, 'male_applicants': 75, 'female_ap

In [241]:
high_school_data.keys()

dict_keys(['high_school', 'Los Angeles', 'Berkeley', 'Davis', 'San Diego', 'Irvine', 'Santa Barbara', 'Riverside', 'Merced', 'Santa Cruz'])

In [242]:
high_school_data['Los Angeles']

{'App': {'All': 202,
  'African American': 7,
  'Hispanic/ Latinx': 22,
  'Asian': 115,
  'White': 44,
  'Domestic Unknown': 9,
  'total_applicants': 202,
  'male_applicants': 90,
  'female_applicants': 104,
  'other_applicants': 0,
  'unknown_gender': 0,
  'Mean GPA': 4.06},
 'Adm': {'All': 25,
  'Asian': 15,
  'White': 5,
  'total_applicants': 25,
  'male_applicants': 9,
  'female_applicants': 14,
  'other_applicants': 0,
  'unknown_gender': 0,
  'Mean GPA': 4.29},
 'Enr': {'All': 10,
  'Asian': 8,
  'total_applicants': 10,
  'male_applicants': 3,
  'female_applicants': 7,
  'other_applicants': 0,
  'unknown_gender': 0,
  'Mean GPA': 4.28}}

In [232]:
s = crud.get_high_school_by_name_and_city(db, 'University High School', '')
for i in s:
    print(i.school_name,  i.city, i.uc_school_name) 

UNIVERSITY HIGH SCHOOL INDIANA Carmel UNIVERSITY HIGH SCHOOL INDIANA150448
UNIVERSITY HIGH SCHOOL Tucson UNIVERSITY HIGH SCHOOL30488
UNIVERSITY HIGH SCHOOL Tolleson UNIVERSITY HIGH SCHOOL30746
UNIVERSITY HIGH SCHOOL Orlando UNIVERSITY HIGH SCHOOL101309
UNIVERSITY HIGH SCHOOL Normal UNIVERSITY HIGH SCHOOL143205
UNIVERSITY HIGH SCHOOL Morgantown UNIVERSITY HIGH SCHOOL490880
INDIANA UNIVERSITY HIGH SCHOOL Bloomington INDIANA UNIVERSITY HIGH SCHOOL150219
UNIVERSITY HIGH SCHOOL Fresno UNIVERSITY HIGH SCHOOL50936
UNIVERSITY HIGH SCHOOL Los Angeles UNIVERSITY HIGH SCHOOL51825
UNIVERSITY HIGH SCHOOL Irvine UNIVERSITY HIGH SCHOOL51984
UNIVERSITY HIGH SCHOOL Waco UNIVERSITY HIGH SCHOOL447320
UNIVERSITY HIGH SCHOOL Spokane Valley UNIVERSITY HIGH SCHOOL481297


IRVINE HIGH SCHOOL Irvine IRVINE HIGH SCHOOL51271
IRVINE HIGH SCHOOL Irvine IRVINE HIGH SCHOOL051271


In [185]:
s.school_name

'WOODBRIDGE HIGH SCHOOL'

In [211]:
import re

In [220]:
# Create or update high school
uc_school_name = 'IRVINE HIGH S1CHOOL051271'
# Remove leading zeros from the numerical part of the school name

if uc_school_name and any(char.isdigit() for char in uc_school_name):
    match = re.search(r'(\d+)$', uc_school_name)
    if match:
        digits = match.group(1)
        stripped_digits = digits.lstrip('0')
        uc_school_name = re.sub(r'\d+$', stripped_digits, uc_school_name)



In [221]:
uc_school_name

'IRVINE HIGH S1CHOOL51271'

In [222]:
from sql_db.process_csv_file import clean_high_school_table

In [226]:
foo = clean_high_school_table(db)

In [224]:
db.close()

In [227]:
foo

['ANDERSON UNION HIGH SCHOOL050095',
 'ANDERSON VALLEY HIGH SCHOOL050355',
 'ANZAR HIGH SCHOOL053133',
 'ARGONAUT HIGH SCHOOL051275',
 'BURBANK INDEPENDENT LEARNING050057',
 'BURNEY JUNIOR-SENIOR HS050419',
 'CAPISTRANO VALLEY CHRSTN SCH053114',
 'CORE BUTTE CHARTER SCHOOL054171',
 'CREATIVE CONNECTIONS ARTS ACAD054565',
 'DURHAM HIGH SCHOOL050790',
 'HAMILTON UNION HIGH SCHOOL051125',
 'HORIZON CHARTER SCHOOL051427',
 'JOHN ADAMS ACADEMY054586',
 'MET SACRAMENTO HIGH SCHOOL053916',
 'PARADISE HIGH SCHOOL052360',
 'PUBLIC SAFETY ACADEMY054757',
 'REX & MARGARET FORTUNE ECHS050394',
 'ROUND VALLEY HIGH SCHOOL050690',
 'SOUTH SUTTER CHARTER SCHOOL054112',
 'TAFT UNION HIGH SCHOOL053485',
 'TOMALES HIGH SCHOOL053510',
 'WEST VALLEY HIGH SCHOOL050688',
 'WOODLAND CHRISTIAN HIGH SCHOOL053850',
 'YESHIVA UNIV HS LOS ANGELES051843',
 'YREKA HIGH SCHOOL053810',
 'AVALON HIGH SCHOOL050180',
 'BASIS MESA030750',
 'BONSALL HIGH SCHOOL054793',
 'CALVIN CHRISTIAN HIGH SCHOOL050876',
 'DESERT RIDGE 